<a href="https://colab.research.google.com/github/sujith2303/ML_From_Scratch/blob/main/7)%20Advanced_Projects/PeftImplementation/peft_implementation_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn as nn
import torch

In [47]:
class MLP(nn.Module):
  def __init__(self,input_dim,output_dim,rank):
    super().__init__()
    self.layer = nn.Linear(input_dim,output_dim)
    self.lora_A = nn.Parameter(torch.zeros(rank,output_dim))
    self.lora_B = nn.Parameter(torch.zeros(input_dim,rank))
    nn.init.normal_(self.lora_A)
    nn.init.normal_(self.lora_B)
  def forward(self,x):
    return self.layer(x) + x @ self.lora_B @ self.lora_A


In [48]:
a = MLP(10,5,2)

In [49]:
a.layer.weight.requires_grad = False
a.layer.bias.requires_grad = False

In [50]:
for name, param in a.named_parameters():
  print(name,param,param.requires_grad)

lora_A Parameter containing:
tensor([[ 0.0704,  0.6876,  2.6952,  0.4876, -1.1048],
        [ 0.8194,  0.8655,  0.4910,  0.5457,  0.7770]], requires_grad=True) True
lora_B Parameter containing:
tensor([[-0.3533, -0.2863],
        [ 0.8812, -0.7112],
        [ 1.3064,  2.4411],
        [-1.3770, -0.3194],
        [-0.2807, -1.1665],
        [ 0.3467, -0.1804],
        [ 0.2115, -1.2617],
        [ 1.8283, -0.0567],
        [ 0.9864,  1.2137],
        [ 1.2741,  0.9932]], requires_grad=True) True
layer.weight Parameter containing:
tensor([[ 0.0696,  0.0941, -0.2601,  0.2813,  0.2521,  0.0981,  0.0656, -0.2960,
         -0.0392,  0.0362],
        [ 0.3032, -0.2225,  0.0082, -0.0442, -0.2522,  0.1042,  0.1358,  0.1444,
          0.0823, -0.2012],
        [ 0.1838, -0.2032, -0.1282, -0.0174, -0.1534,  0.2721, -0.1783,  0.1236,
         -0.2177,  0.0097],
        [-0.2884, -0.1502,  0.0343,  0.2799, -0.3026, -0.0723,  0.0142,  0.0437,
         -0.0480,  0.0276],
        [ 0.2473,  0.1981,  0

In [51]:
a(torch.randn(10))

tensor([ 0.4993, -0.0523, -4.6377, -0.5134,  3.4244], grad_fn=<AddBackward0>)

In [59]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(a.parameters(),lr=5e-5,)

In [53]:
X = torch.ones(1,10)
y = torch.ones(1,5)*35

In [54]:
X,y

(tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 tensor([[35., 35., 35., 35., 35.]]))

In [ ]:
for epoch in range(2000):
  output = a(X)
  l = loss(output,y)
  l.backward()
  optimizer.step()
  print(l)

In [65]:
a(torch.ones(10))

tensor([35.4564, 33.4413, 33.5289, 32.9141, 36.6577], grad_fn=<AddBackward0>)

In [70]:
total = 0
some = 0
for i,j in a.named_parameters():
  if j.requires_grad:
    some+=j.numel()
  total+=j.numel()
print("Trainable parameters:-",some/total)

Trainable parameters:- 0.35294117647058826


In [69]:
some,total

(0, 5)

In [66]:
optimizer.state

defaultdict(dict,
            {Parameter containing:
             tensor([[ 2.0505, -0.1295, -0.4679,  1.9876,  3.3249],
                     [ 0.3922,  4.5915,  5.3520,  0.3100, -2.1798]], requires_grad=True): {'step': tensor(13016.),
              'exp_avg': tensor([[ 46308.0898,  -3028.9744, -25398.5508, -34328.2109, -54617.7969],
                      [ 25982.0488,  -4197.7524, -31718.5371, -17310.4453, -13126.2461]]),
              'exp_avg_sq': tensor([[1.7317e+09, 9.7815e+06, 4.8879e+08, 5.8698e+08, 2.5055e+09],
                      [5.4433e+08, 1.3326e+07, 8.8502e+08, 1.5559e+08, 1.4926e+08]])},
             Parameter containing:
             tensor([[ 0.7369,  0.4212],
                     [ 1.9712, -0.0037],
                     [ 2.3963,  3.1482],
                     [-0.2866,  0.3880],
                     [ 0.8094, -0.4590],
                     [ 1.4367,  0.5270],
                     [ 1.3016, -0.5541],
                     [ 2.9181,  0.6507],
                     [ 2.

# 1.58 bit LLMs